In this file:

The goal is to create a daily excel file, with the top 50 firms (Market Cap > 50m) according to [magic formula investing](https://www.magicformulainvesting.com/) table. This file is then sent to my own personal email every day.

To do this, I will need to perform the following steps:

1. login onto the website (magicformulainvesting.com)
2. Perform the screening: top 50 stocks with Market Cap above 50M USD
3. Web-Scrap the screened stocks - company name, ticker, price date, most recent quarter data
4. Save virtually the dataframe to an excel file
5. Send the excel file through email

This proccess can automatically be repeated daily, through the cloud, on IBM Watson.

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

#to send email:
import smtplib,ssl
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders

import io

##### Your Personal Information:

In [ ]:
#your magicformulainvesting email and password:
email_mf = 'insert_email_here'
password_mf = 'insert_your_password'

#your email gmail account information:
send_from = 'insert_email_here'
password = 'insert_your_password'
send_to = 'insert_email_here'

#### Logging in to the magic formula website website

In [ ]:
login_data = {'Email': email_mf,
'Password': password_mf,
'login': 'Login'}

conditions = {'MinimumMarketCap': '50',
'Select30': 'false',
'stocks': 'Get Stocks'}

In [ ]:
#Because we are loggin in, we need to create a session:
with requests.Session() as s:
    url = 'https://www.magicformulainvesting.com/Account/Logon'
    r = s.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    
    r = s.post(url, data = login_data) #logging in
    
    #Now we need to request the top 50, with market cap above 50
    r = s.post( 'https://www.magicformulainvesting.com/Screening/StockScreening',data = conditions)

##### Now that we are logged in on the desired page, we can start the web scrapping

In [ ]:
#Information we will scrap:
company_name = []
company_ticker = []
market_cap = []
date_price = []
most_recent_quarter_data = []

soup = BeautifulSoup(r.text, 'html.parser')
#appending odd_rows:
for i in soup.find_all('tr', attrs = {'class':"altrow"}):
    company_name.append(i.find_all('td')[0].text.strip())
    company_ticker.append(i.find_all('td')[1].text.strip())
    market_cap.append(i.find_all('td')[2].text.strip())
    date_price.append(i.find_all('td')[3].text.strip())
    most_recent_quarter_data.append(i.find_all('td')[4].text.strip())
    
#For the even rows, the website maintained the class as class = "", which will make us select more information than needed when 
#performing the soup.find_all('tr', attrs = {'class':""}) - so, we'll need to use a conditional statement within the loop:

for i in soup.find_all('tr', attrs = {'class': ""}):
    if len(i.find_all('td')) == 5:
        company_name.append(i.find_all('td')[0].text.strip())
        company_ticker.append(i.find_all('td')[1].text.strip())
        market_cap.append(i.find_all('td')[2].text.strip())
        date_price.append(i.find_all('td')[3].text.strip())
        most_recent_quarter_data.append(i.find_all('td')[4].text.strip())

Now let's turn this information into a pandas dataframe

In [ ]:
df_mf = pd.DataFrame({'company name': company_name, 
              'ticker': company_ticker, 
              'price date': date_price,
              'most recent quarter data': most_recent_quarter_data})
df_mf = df_mf.sort_values('company name').reset_index(drop = True) #sorting by company name

In [ ]:
df_mf

##### Sending email with the excel file:

In [ ]:
#function to export the dataframe to excel format
def export_excel(df):
    with io.BytesIO() as buffer:
        writer = pd.ExcelWriter(buffer)
        df.to_excel(writer)
        writer.save()
        return buffer.getvalue()

In [ ]:
#email content:
subject = 'Magic Formula: Top 50 Stocks'
body = """Good morning,
Please find attached today's top 50 stocks according to the magic formula investing website.
Kind regards,
Zé """
EXPORTERS = {'magic_formula.xlsx': export_excel} #in case you want to export several files, add more to the dictionary

#sending through gmail:
server = 'smtp.gmail.com'
port = 587

In [ ]:
#create funtion to send email:
def send_mail(send_from,send_to,subject,text,files,server,port,password,isTls=True):
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Date'] = formatdate(localtime = True)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))

    part = MIMEBase('application', "octet-stream")
    part.set_payload(open("WorkBook3.xlsx", "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename="WorkBook3.xlsx"')
    msg.attach(part)

    #context = ssl.SSLContext(ssl.PROTOCOL_SSLv3)
    #SSL connection only working on Python 3+
    smtp = smtplib.SMTP(server, port)
    if isTls:
        smtp.starttls()
    smtp.login(send_from,password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()

In [ ]:
#from email.mime.application import MIMEApplication
#from email.mime.multipart import MIMEMultipart
#from email.mime.text import MIMEText
#import smtplib

#Create function to send email:

def send_dataframe(df, isTls = True):
    
    multipart = MIMEMultipart()
    multipart['From'] = send_from
    multipart['To'] = send_to
    multipart['Subject'] = subject
    for filename in EXPORTERS:
        attachment = MIMEApplication(EXPORTERS[filename](df))
        attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(filename)
        multipart.attach(attachment)
    multipart.attach(MIMEText(body, 'html'))
    
    smtp = smtplib.SMTP(server, port)
    if isTls:
        smtp.starttls()
    smtp.login(send_from,password)
    smtp.sendmail(send_from, send_to, multipart.as_string())
    smtp.quit()

In [ ]:
send_dataframe(df = df_mf)

And it is done! Hopefully you learned something.

To automate this process daily, you may use IBM Watson Studio to run your code on the cloud. It's really cool because you do not need to have your pc turned on, or linked to the internet, to have it run every day. Please check the following tuturial on how to automate the process: [Scheduling a Notebook - IBM Watson Studio](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/analyze-data/schedule-task.html)

If you are confused on how this whole process worked out (I know I would be), please check out the below tuturials, with a more basic approach on how to perform web-scrapping, loggin-in to a website, and sending a dataframe through email, all using python.

Have a good one!

Sources:   
Logging in into the website: [Indian Pythonista youtube video](https://www.youtube.com/watch?v=fmf_y8zpOgA)   
Web-Scrapping: [Computer Science web scraping youtube video](https://www.youtube.com/watch?v=Cb_5A6geOUw&list=LL&index=2)   
Sending DataFrame on Excel file through gmail: [snooze92's answer on stackoverflow](https://stackoverflow.com/questions/39999964/send-email-with-a-pandas-dataframe-as-attachment/49298990)